In [1]:
from os import listdir
from os.path import join,isfile
from keras.preprocessing import image
from keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import math
import imutils
import time
from collections import Counter
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import Conv2D,MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation,Dense,Flatten
from pygame import mixer

Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
model = load_model('md_model.h5')
model.compile(loss = "binary_crossentropy",optimizer = Adam(), metrics=['accuracy'],)
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [4]:
face_cascade=cv2.CascadeClassifier('harcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    ret,img=cap.read()
    if not ret:
        continue
    faces=face_cascade.detectMultiScale(img,1.3,5)  
    
    for x,y,w,h in faces:
    
        face_img=img[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(224,224))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        result=model.predict(reshaped)
        label=np.argmax(result,axis=1)[0] 
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        
        
        cv2.putText(img, labels_dict[label], (x, y-10),
          cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()